## `numpy` 101: Axes, universal functions, and broadcasting

In [6]:
import numpy as np

### "Collapsing" functions

`numpy` has lots of useful collapse arrays to numbers:

- `sum` adds up all the elements in an array.
- `prod` multiplies all the elements of an array together.
- `mean` computes the mean of all the elements of an array.
- `median` computes the median of all the elements of an array.
- `var` computes the variance of all the elements of an array.
- `max` returns the largest value of an array.
- `min` returns the smallest value of an array.
- `argmax` gives the index -- in the flattened array -- of the first occurance of an array's maximum value.
- `argmin` gives the index -- in the flattened array -- of the first occurance of an array's minimum value.
- ...

In [7]:
B = np.array(range(1, 16)).reshape(5, 3)
print(B, B.mean(), np.mean(B), np.median(B), np.max(B), np.argmax(B), sep="\n\n")

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]]

8.0

8.0

8.0

15

14


### Collapsing along axes

These same functions, when instructed, can also collapse arrays into smaller arrays.
If `f` is one of the above functions, then `f(B, axis=k)` will apply `f` to collapse the $k$-th axis of `B`:

In [8]:
print(np.sum(B, axis=0), np.mean(B, axis=1), np.argmin(B, axis=0), np.argmax(B, axis=1), sep="\n\n")

[35 40 45]

[ 2.  5.  8. 11. 14.]

[0 0 0]

[2 2 2 2 2]


If $j \neq k$, then the $j$-th dimension of `f(B, axis=k)` is the same as that of `B`. The $k$-th dimension of `f(B, axis=k)` is $1$. (The $j$-th dimension of `B` is `B.shape[j]`.)

Axis 0 is the row axis. Collapsing along axis $0$ means collapsing all the rows down to a single row. Similarly for axis $1$ and columns.

### Repeating along an axis

`np.repeat(C, n, axis=k)` will repeat `C` `n` times along its `k`-th axis.

In [9]:
u = np.array([[1, 2, 3]])
v = 2*np.array(range(-4, 8)).reshape(4, 3)
w = np.array(range(4, -1, -1)).reshape(5, 1)

print(u, np.repeat(u, 3, axis=0), np.repeat(u, 4, axis=1), sep="\n\n")

[[1 2 3]]

[[1 2 3]
 [1 2 3]
 [1 2 3]]

[[1 1 1 1 2 2 2 2 3 3 3 3]]


### Universal functions

In `numpy`, Basic arithmetic operations on arrays (matrices) are performed component-wise...

In [10]:
x = np.array([1, 2, 3])
y = np.array([4, 10, 6])
print(x + y, x*y, x/y, x**y, sep="\n\n")

[ 5 12  9]

[ 4 20 18]

[0.25 0.2  0.5 ]

[   1 1024  729]


...and obey certain size compatibility constraints.

In [11]:
z = np.array([1, -1, 0, 1])
try:
    x * z
except ValueError as e:
    print(f"ValueError: {e}")

ValueError: operands could not be broadcast together with shapes (3,) (4,) 


`numpy` has plenty of standard mathematical functions built in. These also operate component-wise on arrays:

In [12]:
A = np.array([x, y])
print(np.exp2(x), np.exp(x), np.sqrt(x), np.square(x), sep="\n\n")

[2. 4. 8.]

[ 2.71828183  7.3890561  20.08553692]

[1.         1.41421356 1.73205081]

[1 4 9]


In `numpy`, functions that operate componentwise on arrays of arbitrary size are called **universal functions** (`ufunc`s).

### Broadcasting

In linear algebra class, it's common to multiply a vector matrix by a scalar:

In [13]:
print(A, 3*A, sep="\n\n")

[[ 1  2  3]
 [ 4 10  6]]

[[ 3  6  9]
 [12 30 18]]


What's not so common is to add a number to a vector or matrix:

In [14]:
print(x, x + 1, A, A - 1, sep="\n\n")

[1 2 3]

[2 3 4]

[[ 1  2  3]
 [ 4 10  6]]

[[0 1 2]
 [3 9 5]]


`numpy` knows that adding a number to an array means adding that number to every component of an array.
This is a special case of `numpy`'s scheme, called **broadcasting**, for applying **universal functions of several variables** --- like `+`, `*`, `/`, and `**` --- to arrays with different sizes.

In [15]:
print(u, v, u + v, u, w, u * w, sep="\n\n")

[[1 2 3]]

[[-8 -6 -4]
 [-2  0  2]
 [ 4  6  8]
 [10 12 14]]

[[-7 -4 -1]
 [-1  2  5]
 [ 5  8 11]
 [11 14 17]]

[[1 2 3]]

[[4]
 [3]
 [2]
 [1]
 [0]]

[[ 4  8 12]
 [ 3  6  9]
 [ 2  4  6]
 [ 1  2  3]
 [ 0  0  0]]


- We added the $1 \times 3$ array `u` to the $4\times 3$ array `v`, producing a $4\times 3$ array.
- We multiplied the $1 \times 3$ array `u` to the $5\times 1$ array `v`, producing a $5\times 3$ array.

Here are the rules:

- `A + B` is defined **if and only if**
  <p style="text-align: center; padding-bottom: 1em">for all `j`, either `A.shape[j] == B.shape[j]` or one of `A.shape[j]`, `B.shape[j]` is `1`,</p>
  in which case `(A + B).shape[j] == np.max(A.shape[j], B.shape[j])`.

- If `1 == A.shape[j] < B.shape[j]`, then `A` is duplicated along the `j`-th axis `B.shape[j]` times, and symmetrically if `1 == B.shape[j] < A.shape[j]`. The new matrices are then added as usual.

**Note:** `numpy` doesn't literally duplicte rows/columns, creating a large, redundant matrix.
That would be inefficient. Nonetheless, I find this is a useful way to visualize how broadcasting works.

In [16]:
uu = np.repeat(u, 4, axis=0)
print(u, v, u + v, uu, v, uu + v, sep="\n\n")

[[1 2 3]]

[[-8 -6 -4]
 [-2  0  2]
 [ 4  6  8]
 [10 12 14]]

[[-7 -4 -1]
 [-1  2  5]
 [ 5  8 11]
 [11 14 17]]

[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]

[[-8 -6 -4]
 [-2  0  2]
 [ 4  6  8]
 [10 12 14]]

[[-7 -4 -1]
 [-1  2  5]
 [ 5  8 11]
 [11 14 17]]


In [17]:
print(u.T, v.T, u.T + v.T, uu.T, v.T, uu.T + v.T, sep="\n\n")

[[1]
 [2]
 [3]]

[[-8 -2  4 10]
 [-6  0  6 12]
 [-4  2  8 14]]

[[-7 -1  5 11]
 [-4  2  8 14]
 [-1  5 11 17]]

[[1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]]

[[-8 -2  4 10]
 [-6  0  6 12]
 [-4  2  8 14]]

[[-7 -1  5 11]
 [-4  2  8 14]
 [-1  5 11 17]]


### Test your understanding

1. Arrange the numbers $1$ through $35$ in a $5 \times 7$ matrix, $A$.
   
   1. Compute the mean of all the entries of $A$.
   2. Compute the row-means of $A$, i.e., the vector of length $5$ whose $i$-th entry is the mean of the $i$-th row of $A$.
   3. Subtract the row-means from their rows, so that the row-means of the resulting difference are all $0$. (We call this **centering** the rows of $A$.)
   4. Divide each centered row by its standard deviation (`np.std`). Confirm that the rows of the resulting matrix all have variance (`np.var`) $1$.

2. Let $B$ be a $6\times 6$ matrix whose entries are drawn independently at random from the standard normal distribution (`numpy.random.normal`). In each column of $B$, find the entry closest to $0.5$, and its row index. (Hint: `np.abs` and `np.argmin`)

3. Let $C$ be a $5\times 10$ matrix whose entries are drawn independently at random from the standard uniform distribution (`numpy.random.uniform`). In each row of $C$, find the entries furthest from $0.0$, $0.33$, $0.66$, and $1.0$, as well as their column indices. (Hint: `np.abs` and `np.argmax`)